In [51]:
import json
import pprint

import numpy as np
import pandas as pd

from fastembed import TextEmbedding
from qdrant_client import QdrantClient, models

In [4]:
model_handle = 'jinaai/jina-embeddings-v2-small-en'
model = TextEmbedding(model_name=model_handle)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

onnx/model.onnx:   0%|          | 0.00/130M [00:00<?, ?B/s]

In [16]:
# Q1
query='I just discovered the course. Can I join now?'
query_embedding=list(model.embed([query]))[0]
query_embedding.min()

np.float64(-0.11726373885183883)

In [17]:
np.linalg.norm(embedding)

np.float64(1.0000000000000002)

In [30]:
# Q2
doc = 'Can I still join the course after the start date?'
doc_embedding=list(model.embed([doc]))[0]
doc_embedding @ query_embedding


np.float64(0.9008528895674548)

In [15]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [31]:
# Q3
documents_embeddings=list(model.embed([doc['text'] for doc in documents]))
V=np.array(documents_embeddings)
print(V.shape)
print(V @ query_embedding)
(V @ query_embedding).argmax()

(5, 512)
[0.76296845 0.81823782 0.80853974 0.71330788 0.73044992]


np.int64(1)

In [41]:
# Q4

# question 0 is much more similar to  the query

[ doc.update(full_text = doc['question'] + ' ' + doc['text']) for  doc in documents ];

documents_embeddings=list(model.embed([doc['full_text'] for doc in documents]))
V=np.array(documents_embeddings)
print(V.shape)
print(V @ query_embedding)
(V @ query_embedding).argmax()


(5, 512)
[0.85145432 0.8436594  0.84082872 0.77551577 0.80860079]


np.int64(0)

In [46]:
# Q5
min_embed=1024
model_min_embed={}
for model in TextEmbedding.list_supported_models():
    if model['dim']<min_embed:
        min_embed=model['dim']
        model_min_embed=model

pprint.pp(model_min_embed)

{'model': 'BAAI/bge-small-en',
 'sources': {'hf': 'Qdrant/bge-small-en',
             'url': 'https://storage.googleapis.com/qdrant-fastembed/BAAI-bge-small-en.tar.gz',
             '_deprecated_tar_struct': True},
 'model_file': 'model_optimized.onnx',
 'description': 'Text embeddings, Unimodal (text), English, 512 input tokens '
                'truncation, Prefixes for queries/documents: necessary, 2023 '
                'year.',
 'license': 'mit',
 'size_in_GB': 0.13,
 'additional_files': [],
 'dim': 384,
 'tasks': {}}


In [47]:
# Q6

model_handle='BAAI/bge-small-en'

import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()



In [48]:
documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [50]:
'''
docker pull qdrant/qdrant

docker run -it -p 6333:6333 -p 6334:6334 \
   -v "$(pwd)/qdrant_storage:/qdrant/storage:z" \
   qdrant/qdrant
''';

In [52]:
client = QdrantClient("http://localhost:6333")

In [53]:
# Define the collection name
collection_name = "zoomcamp-rag"

# Create the collection with specified vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=min_embed,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)


points = []
id = 0

for course in documents_raw:
    for doc in course['documents']:

        point = models.PointStruct(
            id=id,
            vector=models.Document(text=doc['question'] + ' ' + doc['text'], 
                                   model=model_handle),
            payload={
                "text": doc['text'],
                "section": doc['section'],
                "course": course['course']
            } #save all needed metadata fields
        )
        points.append(point)

        id += 1



In [54]:
client.upsert(
    collection_name=collection_name,
    points=points
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [56]:
def search(query, limit=1):

    results = client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model=model_handle 
        ),
        limit=limit, # top closest matches
        with_payload=True, #to get metadata in the results
        with_vectors=True
    )

    return results

result=search(query)
result.points[0].score

0.87031734